In [180]:
from visual_behavior.visualization.qc import session_plots as sp
from visual_behavior.visualization.qc import container_plots as cp
from visual_behavior.visualization.qc import data_loading as dl
from visual_behavior.visualization.qc import data_processing as dp
import visual_behavior.utilities as vbu

from visual_behavior.translator.foraging2 import data_to_change_detection_core
from visual_behavior.translator.core import create_extended_dataframe
from visual_behavior.change_detection.trials import summarize
import visual_behavior.database as db

%matplotlib inline
%widescreen
%standard_imports

imported:
	os
	pandas as pd
	numpy as np
	matplotlib.pyplot as plt


## get ophys experiment table

In [181]:
from allensdk.brain_observatory.behavior.behavior_project_cache import BehaviorProjectCache as bpc
manifest_file = r"//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020_cache/covid_cache/manifest_v2.json"
cache = bpc.from_lims(manifest=manifest_file)
all_experiments = cache.get_experiment_table() 
all_experiments = all_experiments.reset_index()

In [182]:
len(all_experiments)

3030

## functions

In [183]:
def oeid_to_uuid(oeid):
    uuid = db.convert_id({'ophys_experiment_id':oeid},'behavior_session_uuid')
    # there are a couple cases where LIMS doesn't have the UUID. Dig into the PKL file to get it
    if pd.isnull(uuid):
        try:
            osid = db.convert_id({'ophys_experiment_id':oeid},'ophys_session_id')
            pkl_path = get_pkl_path_from_osid(osid)
            data = pd.read_pickle(pkl_path)
            uuid = data['session_uuid']
        except Exception as e:
            print('could not find UUID for oeid {}'.format(oeid))
            uuid = None
    return uuid

def get_pkl_path_from_osid(osid):
    wkf = db.get_well_known_files(osid).set_index('name')
    return ''.join([wkf.loc['StimulusPickle']['storage_directory'], wkf.loc['StimulusPickle']['filename']])

def get_stage_from_mongo(uuid):
    vb = db.Database('visual_behavior_data')
    stats = vb.behavior_data['summary'].find_one({'behavior_session_uuid':uuid})
    vb.close()
    if stats is None or ('error_on_load' in stats.keys() and stats['error_on_load'] == 1):
        return None
    else:
        return stats['stage']

def get_date_from_mongo(uuid):
    vb = db.Database('visual_behavior_data')
    stats = vb.behavior_data['summary'].find_one({'behavior_session_uuid':uuid})
    vb.close()
    if stats is None or ('error_on_load' in stats.keys() and stats['error_on_load'] == 1):
        return None
    else:
        return stats['startdatetime']
    
def get_container_info(ophys_experiment_id):
    container_id = all_experiments.query('ophys_experiment_id == @ophys_experiment_id')['container_id'].iloc[0]
    return dp.ophys_container_info_df(container_id)

def get_lims_stage_name(ophys_experiment_id):
    container_info = get_container_info(ophys_experiment_id)
    try:
        return container_info.query('ophys_experiment_id == @ophys_experiment_id')['stage_name_lims'].iloc[0]
    except:
        pass

def get_mtrain_stage_name(ophys_experiment_id):
    container_info = get_container_info(ophys_experiment_id)
    try:
        return container_info.query('ophys_experiment_id == @ophys_experiment_id')['stage_name_mtrain'].iloc[0]
    except:
        pass
    
    
def rewrite_record(uuid):
    try:
        pkl_path = db.get_pkl_path(uuid)
        data = pd.read_pickle(pkl_path)
        vb = db.Database('visual_behavior_data')
        core_data = data_to_change_detection_core(data)
        trials = create_extended_dataframe(**core_data).drop(columns=['date', 'LDT_mode'])
        summary = summarize.session_level_summary(trials).iloc[0].to_dict()
        summary.update({'error_on_load':0})
        db.update_or_create(
            vb['behavior_data']['summary'],
            db.simplify_entry(summary),
            ['behavior_session_uuid'],
            force_write=False
        )
        vb.close()
    except:
        print('failed on uuid {}'.format(uuid))

## filter on sessions with visual behavior project codes

In [184]:
visb_experiments = all_experiments[all_experiments.project_code.isin(['VisualBehavior','VisualBehaviorTask1B'])].copy()
len(visb_experiments)

827

## add columns with date and session from mongo (which comes directly from PKL files)

In [185]:
%%time
visb_experiments['uuid'] = visb_experiments['ophys_experiment_id'].map(lambda oeid:oeid_to_uuid(oeid))

visb_experiments['pkl_stage_name'] = visb_experiments['uuid'].map(lambda uuid:get_stage_from_mongo(uuid))
visb_experiments['mongo_date'] = visb_experiments['uuid'].map(lambda uuid:get_date_from_mongo(uuid))

visb_experiments['lims_stage_name'] = visb_experiments['ophys_experiment_id'].map(lambda oeid:get_lims_stage_name(oeid))
visb_experiments['mtrain_stage_name'] = visb_experiments['ophys_experiment_id'].map(lambda oeid:get_mtrain_stage_name(oeid))

CPU times: user 1min 35s, sys: 9.56 s, total: 1min 45s
Wall time: 3min 36s


In [191]:
visb_experiments.sample(5)

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,uuid,pkl_stage_name,mongo_date,lims_stage_name,mtrain_stage_name,problem_type,desired_LIMS_stage_name
459,779347929,778861532,778991399,814796698,VisualBehavior,failed,failed,20181114_412364_,OPHYS_1_images_A,CAM2P.4,2018-11-14 17:23:02.000000,756132689,744935649,M,121.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375,VISp,NaN,117e82ec-5ff9-4410-a7b8-e155cde0fbb9,OPHYS_1_images_A,2018-11-14 18:48:21.613,OPHYS_1_images_A,OPHYS_1_images_A,None,None
2463,836260147,835929094,836042327,830457573,VisualBehavior,failed,failed,20190312_433245_5_images_B,OPHYS_5_images_B_passive,CAM2P.5,2019-03-12 18:44:12.000000,805338209,795522276,F,127.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375,VISp,NaN,4e05e9b1-bcb1-4ddf-adca-7fb4d9c310b1,OPHYS_5_images_B_passive,2019-03-12 19:04:42.628,OPHYS_5_images_B_passive,OPHYS_5_images_B_passive,None,None
469,791119849,790915236,790951764,814796698,VisualBehavior,failed,passed,20181205_412364_2imagesA_passive,OPHYS_2_images_A_passive,CAM2P.4,2018-12-05 20:54:41.000000,756132689,744935649,M,142.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375,VISp,NaN,8bb2c313-25da-4f12-bccd-bb5ad1ca928d,OPHYS_2_images_A_passive,2018-12-05 21:46:12.463,OPHYS_2_images_A_passive,OPHYS_2_images_A_passive,None,None
601,906876035,906299686,906437131,895259658,VisualBehaviorTask1B,failed,failed,20190715_460746_5ImagesA,OPHYS_5_images_A_passive,CAM2P.5,2019-07-15 15:18:38.000000,866869038,859036458,F,124.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375,VISp,NaN,440dacce-bf94-4404-a34c-21aeee86dcd8,OPHYS_5_images_A_passive,2019-07-15 15:46:48.112,OPHYS_5_images_A_passive,OPHYS_5_images_A_passive,None,None
824,855582981,855353286,855446545,845588020,VisualBehavior,container_qc,passed,20190422_445270_5imagesBPassive,OPHYS_5_images_B_passive,CAM2P.3,2019-04-22 19:33:25.000000,828775962,820871408,M,109.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375,VISp,NaN,91335912-88ca-41fd-aad7-d5e82606f5dc,OPHYS_5_images_B_passive,2019-04-22 19:51:54.729,OPHYS_5_images_B_passive,OPHYS_5_images_B_passive,None,None


## load Kate's list of mismatched sessions

In [192]:
kate_mismatch_df = pd.read_csv('/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/stage_name_mismatches_identified_by_kate_03.21.2020.csv')

## search for mismatches in stage name

In [193]:
my_mismatches = visb_experiments[visb_experiments['session_type'] != visb_experiments['pkl_stage_name']]

cols_to_show = [
    'ophys_experiment_id',
    'ophys_session_id',
    'container_id',
    'project_code',
    'container_workflow_state',
    'experiment_workflow_state',
    'session_name',
    'session_type',
    'pkl_stage_name',
    'lims_stage_name',
]
my_mismatches[cols_to_show]

,ophys_experiment_id,ophys_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,pkl_stage_name,lims_stage_name
70,868245889,867455174,898954413,VisualBehavior,failed,failed,20190514_448897_7RF,NaN,None,OPHYS_7_receptive_field_mapping
74,872434950,871522846,898954413,VisualBehavior,failed,failed,20190520_448897_7RF_Map,NaN,None,OPHYS_7_receptive_field_mapping
86,996046242,995978667,991913453,VisualBehaviorTask1B,completed,passed,20191223_476970_7RF,OPHYS_7_receptive_field_mapping,None,OPHYS_7_receptive_field_mapping
93,797257159,796608428,791352433,VisualBehavior,container_qc,passed,20181217_412036_7_RF_map,full_field_test,None,OPHYS_7_receptive_field_mapping
103,834279512,833705055,825183217,VisualBehavior,container_qc,passed,20190307_429952_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping
116,885061426,884352232,876693090,VisualBehavior,container_qc,passed,20190611_449441_7RF_Map,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping
126,995382231,995074586,991913064,VisualBehaviorTask1B,holding,failed,20191218_491060_7RF,OPHYS_7_receptive_field_mapping,None,OPHYS_7_receptive_field_mapping
127,995623020,995558073,991913064,VisualBehaviorTask1B,holding,qc,20191220_491060_7RF_retake,OPHYS_7_receptive_field_mapping,None,OPHYS_7_receptive_field_mapping
152,881881193,881236651,869781307,VisualBehavior,container_qc,passed,20190605_450471_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping
223,872432459,871534712,863992815,VisualBehavior,container_qc,passed,20190520_448900_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping


## Document different types of stage naming issues

In [194]:
visb_experiments['problem_type'] = None
cols_to_show += ['problem_type']

In [195]:
visb_experiments['desired_LIMS_stage_name'] = None
cols_to_show += ['desired_LIMS_stage_name']

### RF mapping sessions with incorrect LIMS name
There are three names for RF mapping sessions. The naming convention should be unified.

In [196]:
incorrect_rf_mapping_session_types = [
    'VisCodingTargetedMovieClips',
    'full_field_test'
]
rf_mapping_misnamed = visb_experiments[
    (visb_experiments['session_type'].isin(incorrect_rf_mapping_session_types))
]
visb_experiments.loc[rf_mapping_misnamed.index.values,'problem_type'] = 'rf_mapping_incorrect_lims_name'
visb_experiments.loc[rf_mapping_misnamed.index.values,'desired_LIMS_stage_name'] = 'OPHYS_7_receptive_field_mapping'
visb_experiments.query('problem_type == "rf_mapping_incorrect_lims_name"')[cols_to_show]

,ophys_experiment_id,ophys_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,pkl_stage_name,lims_stage_name,problem_type,desired_LIMS_stage_name
93,797257159,796608428,791352433,VisualBehavior,container_qc,passed,20181217_412036_7_RF_map,full_field_test,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
103,834279512,833705055,825183217,VisualBehavior,container_qc,passed,20190307_429952_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
116,885061426,884352232,876693090,VisualBehavior,container_qc,passed,20190611_449441_7RF_Map,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
152,881881193,881236651,869781307,VisualBehavior,container_qc,passed,20190605_450471_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
223,872432459,871534712,863992815,VisualBehavior,container_qc,passed,20190520_448900_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
397,899082454,898944825,902748564,VisualBehavior,container_qc,passed,20190702_456564_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
415,800034812,799931731,800219216,VisualBehavior,failed,failed,20181220_408224_7RF,full_field_test,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
424,798598354,793204176,814797471,VisualBehavior,failed,failed,20181211_409258_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
448,856095742,855753717,848390542,VisualBehavior,container_qc,passed,20190423_447663_7RF,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping
470,791450782,791121085,814796698,VisualBehavior,failed,failed,20181206_412364_7RFMapping,VisCodingTargetedMovieClips,None,OPHYS_7_receptive_field_mapping,rf_mapping_incorrect_lims_name,OPHYS_7_receptive_field_mapping


### RF mapping sessions without a session type in LIMS

In [197]:
rf_mapping_session_types = [
    'OPHYS_7_receptive_field_mapping',
    'VisCodingTargetedMovieClips',
    'full_field_test'
]
rf_mapping_mismatches = visb_experiments[
    (visb_experiments['session_name'].str.contains('7RF'))
    &(pd.isnull(visb_experiments['pkl_stage_name']))
    &(pd.isnull(visb_experiments['session_type']))
]
visb_experiments.loc[rf_mapping_mismatches.index.values,'problem_type'] = 'rf_mapping_missing_lims_name'
visb_experiments.loc[rf_mapping_mismatches.index.values,'desired_LIMS_stage_name'] = 'OPHYS_7_receptive_field_mapping'
visb_experiments.query('problem_type == "rf_mapping_missing_lims_name"')[cols_to_show]

,ophys_experiment_id,ophys_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,pkl_stage_name,lims_stage_name,problem_type,desired_LIMS_stage_name
70,868245889,867455174,898954413,VisualBehavior,failed,failed,20190514_448897_7RF,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
74,872434950,871522846,898954413,VisualBehavior,failed,failed,20190520_448897_7RF_Map,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
393,894725344,894247390,902748564,VisualBehavior,container_qc,failed,20190625_456564_7RF,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
396,897374358,896264501,902748564,VisualBehavior,container_qc,failed,20190628_456564_7RF,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
605,910199584,909228684,895259658,VisualBehaviorTask1B,failed,passed,20190719_460746_7RF_map,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
823,853965797,853744857,845588020,VisualBehavior,container_qc,failed,20190417_445270_7RF,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
831,862016695,861710170,845588020,VisualBehavior,container_qc,failed,20190502_445270_7RF,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
833,864936381,864811045,845588020,VisualBehavior,container_qc,failed,20190508_445270_7RF,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
1356,914062473,913565955,904629174,VisualBehaviorTask1B,failed,passed,20190729_462887_7RF_passive,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping
1605,895418485,894835974,890047007,VisualBehavior,container_qc,failed,20190626_456916_7RF_Map,NaN,None,OPHYS_7_receptive_field_mapping,rf_mapping_missing_lims_name,OPHYS_7_receptive_field_mapping


### session type doesn't match PKL stage name

In [198]:
pkl_mismatches = visb_experiments[
    (pd.notnull(visb_experiments['pkl_stage_name']))
    &(visb_experiments['session_type'] != visb_experiments['pkl_stage_name'])
]
visb_experiments.loc[pkl_mismatches.index.values,'problem_type'] = 'pkl_mismatch'
visb_experiments.loc[pkl_mismatches.index.values,'desired_LIMS_stage_name'] = visb_experiments.loc[pkl_mismatches.index.values]['pkl_stage_name']
visb_experiments.query('problem_type == "pkl_mismatch"')[cols_to_show]

,ophys_experiment_id,ophys_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,pkl_stage_name,lims_stage_name,problem_type,desired_LIMS_stage_name
414,800034837,798782540,800219216,VisualBehavior,failed,failed,20181219_408224_6_images_B,OPHYS_6_images_B,OPHYS_2_images_A_passive,OPHYS_1_images_B,pkl_mismatch,OPHYS_2_images_A_passive
608,972666750,972022565,966316564,VisualBehaviorTask1B,failed,created,20191023_479835_session1b,NaN,OPHYS_1_images_B,OPHYS_1_images_B,pkl_mismatch,OPHYS_1_images_B
1367,901557228,898849158,890047007,VisualBehavior,container_qc,qc,20190702_456916_7RF_MAP_retake,NaN,OPHYS_3_images_E,OPHYS_3_images_E,pkl_mismatch,OPHYS_3_images_E
1947,1005123562,1004988945,1000740435,VisualBehaviorTask1B,failed,qc,486738_20200204_4imagesA,OPHYS_3_images_B,OPHYS_4_images_A,OPHYS_4_images_A,pkl_mismatch,OPHYS_4_images_A
2527,974948847,974659900,968890932,VisualBehavior,failed,failed,20191031_483803_3imagesA,OPHYS_1_images_A,OPHYS_3_images_A,OPHYS_3_images_A,pkl_mismatch,OPHYS_3_images_A


In [201]:
get_lims_stage_name(800034837)

'OPHYS_1_images_B'

## check against Kate's list

In [172]:
def in_kate_list(oeid):
    return oeid in kate_mismatch_df['ophys_experiment_id'].tolist()

def in_my_list(oeid):
    return oeid in visb_experiments[pd.notnull(visb_experiments['problem_type'])]['ophys_experiment_id'].tolist()

In [173]:
visb_experiments['in_kate_list'] = visb_experiments['ophys_experiment_id'].map(lambda oeid: in_kate_list(oeid))
visb_experiments[pd.notnull(visb_experiments['problem_type'])]['in_kate_list'].value_counts()

True     38
False    23
Name: in_kate_list, dtype: int64

In [174]:
kate_mismatch_df['in_my_list'] = kate_mismatch_df['ophys_experiment_id'].map(lambda oeid: in_my_list(oeid))
kate_mismatch_df['in_my_list'].value_counts()

True     38
False    25
Name: in_my_list, dtype: int64

### the values that are in Kate's list but not mine are due to mismatches in mtrain. Since mtrain is not the database of record, I think we can ignore these

In [175]:
kate_mismatch_df.query('in_my_list == False')

,ophys_container_id,ophys_experiment_id,ophys_session_id,experiment_workflow_state,stage_name_lims,stage_name_mtrain,in_my_list
0,978206330,990662141,990457777,passed,OPHYS_7_receptive_field_mapping,NaN,False
1,969421516,967007345,966317904,passed,OPHYS_7_receptive_field_mapping,NaN,False
2,968451510,974384470,974019098,passed,OPHYS_3_images_A,NaN,False
3,968451510,982860964,982454932,failed,OPHYS_7_receptive_field_mapping,NaN,False
4,957570596,972679204,972289789,passed,OPHYS_7_receptive_field_mapping,NaN,False
5,957366099,970165639,969890588,passed,OPHYS_7_receptive_field_mapping,NaN,False
6,957361481,969809672,969352777,passed,OPHYS_7_receptive_field_mapping,NaN,False
7,957024596,967873362,967401487,passed,OPHYS_7_receptive_field_mapping,NaN,False
8,947838486,965930948,965338397,failed,OPHYS_7_receptive_field_mapping,NaN,False
9,941373529,961283840,961111597,failed,OPHYS_7_receptive_field_mapping,NaN,False


In [176]:
visb_experiments.query('ophys_experiment_id == 875729856')

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,uuid,pkl_stage_name,mongo_date,lims_stage_name,mtrain_stage_name,problem_type,in_kate_list,desired_LIMS_stage_name
227,875729856,875005830,875020233,863992815,VisualBehavior,container_qc,failed,20190524_448900_3imagesA,OPHYS_2_images_A_passive,CAM2P.4,2019-05-24 15:13:38.000000,838753709,830901424,M,130.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,[Ai94(TITL-GCaMP6s)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,6b74f544-44d7-4029-89fa-1d9c95910538,OPHYS_2_images_A_passive,2019-05-24 16:08:02.352,OPHYS_2_images_A_passive,OPHYS_3_images_A,None,True,None


In [177]:
visb_experiments.query('ophys_experiment_id == 788490510')

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,uuid,pkl_stage_name,mongo_date,lims_stage_name,mtrain_stage_name,problem_type,in_kate_list,desired_LIMS_stage_name
1202,788490510,787661032,788017709,782536745,VisualBehavior,container_qc,passed,20181129_403491_6imagesB,OPHYS_6_images_B,CAM2P.5,2018-11-29 16:20:30.000000,730814230,722884882,F,181.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375,VISp,NaN,bdc41492-1797-4c91-81ba-18fc0a25d238,OPHYS_6_images_B,2018-11-29 17:32:57.281,OPHYS_6_images_B,OPHYS_7_receptive_field_mapping,None,True,None


## save out full list of sessions with issues

In [178]:
filename = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/stage_name_mismatches_03.24.2020.csv'
visb_experiments[pd.notnull(visb_experiments['problem_type'])].to_csv(filename,index=False)

In [179]:
len(visb_experiments[pd.notnull(visb_experiments['problem_type'])])

61